# Transcribe Recordings

paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
import os
import docx

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *
from lib_henryk.utils import *

from lib_henryk.recordings import transcription
from lib_henryk.recordings import recordings

## Params

In [13]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
# and picking up the token from the URL
WEBHOOKS_TOKEN_ID = '69109a84-4073-4747-839d-81df514a77c0'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [19]:
# get fresh stats directly from the recordings
df = recordings.get_recordings_info(path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA)

2024-05-20 02:17:57 - INFO  - get_recordings_info - retrieved 508 files
processing 508 audio files |██████████████████████████████████████████████████| 100.0% 


## Run Transcriptions

In [23]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    verbose=False
)

submitted 0 recordings to the transcription service GoodTape


## Fetch Transcriptions JSONs

In [7]:
# need to wait until transcriptions are available
exit_cell()

StopExecution: stopped

In [11]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json(
    df=df, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
    to_doc=True,
    to_txt=True,
    verbose=False,
)

# process json to doc

found 1 requests to fetch
retrieved transcription [0]: 4c9b51f7-ca18-41fe-aa60-9bea29f060cc -> file [86]: Henryk 2024-05-17 Tatuś opowiada o zabawie z Hanią Tygrys Szab...
1 transcriptions were retrieved
processing 508 transcriptions |██████████████████████████████████████████████████| 100.0% done.
1 new transcriptions were processed, there are 503 txt and 503 doc transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [24]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcription.get_verified_transcriptions(
    df=df, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
)

# save transcription log
df_transcriptions_to_write.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

verifying 508 transcriptions |██████████████████████████████████████████████████| 100.0% done.
there are 508 processed and valid transcriptions
*** all 508 recordings were transcribed ***
